# Coding agents from scratch

In [182]:
import os
from openai import OpenAI
from dotenv import load_dotenv
from typing import List, Dict, Callable

In [183]:
env = load_dotenv()
api_key: str = os.environ.get("OPENAI_API_KEY")

In [184]:
client = OpenAI(api_key=api_key)

In [185]:
messages = [
    {
        "role": "developer",
        "content": [{
            "type": "text",
            "text": "You are a helpful assistant who answers in the style of a Southern Belle"
    }]},
    {
        "role": "user",
        "content": [{
            "type": "text",
            "text": "What's the capital of France?"
    }]
    },
]

In [186]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    max_tokens=1000)

In [187]:
response.choices[0].message.content

'Well, sugar, the capital of France is just as lovely as a spring bloom in a garden! It’s Paris, the City of Light. Now, isn’t that just delightful? 💖'

## Agentic workflows

### Prompt-Chaining: Decompose a task into sequential subtasks, where each step builds on previous results

#### Example 1: Chain workflow for structured data extraction and formatting
Each step progressively transforms raw text into a formatted table

In [188]:


data_processing_steps = [
    """Extract only the numerical values and their associated metrics from the text.
    Format each as 'value: metric' on a new line.
    Example format:
    92: customer satisfaction
    45%: revenue growth""",
    
    """Convert all numerical values to percentages where possible.
    If not a percentage or points, convert to decimal (e.g., 92 points -> 92%).
    Keep one number per line.
    Example format:
    92%: customer satisfaction
    45%: revenue growth""",
    
    """Sort all lines in descending order by numerical value.
    Keep the format 'value: metric' on each line.
    Example:
    92%: customer satisfaction
    87%: employee satisfaction""",
    
    """Format the sorted data as a markdown table with columns, without tags or enclosing quotes:
    | Metric | Value |
    |:--|--:|
    | Customer Satisfaction | 92% |"""
]

In [209]:
class Chainer:
    def __init__(self, system_prompt, steps, inp, model, api_key):
        self.client = OpenAI(api_key=api_key)
        self.steps = steps
        self.inp = inp
        self.model = model
        self.system_prompt = {
            "role": "developer",
            "content": [{
                "type": "text",
                "text": system_prompt
            }]
        }


    def chain(self, verbose=False):
        chain = [self.system_prompt]
        current_input = self.inp
        for ix, step in enumerate(self.steps):            
            new_step = {
                "role": "user",
                "content":
                [{
                    "type": "text",
                    "text":f"{step}\nInput: {current_input}\n"
                }]
            }
            chain.append(new_step)
            response = self.client.chat.completions.create(
                model=self.model,
                messages=chain,
                max_completion_tokens=3000)
            response_content = response.choices[0].message.content
            
            # Add the assistant's response to the chain
            assistant_response = {
                "role": "assistant",
                "content": [{
                    "type": "text",
                    "text": response_content
                }]
            }
            chain.append(assistant_response)
            if verbose:
                print(f"STEP: {ix}")
                print(response_content)
                print("="*36)
            current_input = response_content
        return response_content

In [210]:
system_prompt = "You are a helpful assistant specializing in marketing analysis."

report = """
Q3 Performance Summary:
Our customer satisfaction score rose to 92 points this quarter.
Revenue grew by 45% compared to last year.
Market share is now at 23% in our primary market.
Customer churn decreased to 5% from 8%.
New user acquisition cost is $43 per user.
Product adoption rate increased to 78%.
Employee satisfaction is at 87 points.
Operating margin improved to 34%.
"""

In [218]:
chainer1 = Chainer(system_prompt=system_prompt, steps=data_processing_steps, inp=report, model="gpt-4o", api_key=api_key)
out = chainer1.chain(verbose=True)

STEP: 0
92: customer satisfaction  
45%: revenue growth  
23%: market share  
5%: customer churn  
43: new user acquisition cost  
78%: product adoption rate  
87: employee satisfaction  
34%: operating margin
STEP: 1
92%: customer satisfaction  
45%: revenue growth  
23%: market share  
5%: customer churn  
4300%: new user acquisition cost  
78%: product adoption rate  
87%: employee satisfaction  
34%: operating margin
STEP: 2
4300%: new user acquisition cost  
92%: customer satisfaction  
87%: employee satisfaction  
78%: product adoption rate  
45%: revenue growth  
34%: operating margin  
23%: market share  
5%: customer churn
STEP: 3
| Metric | Value |
|:--|--:|
| New User Acquisition Cost | 4300% |
| Customer Satisfaction | 92% |
| Employee Satisfaction | 87% |
| Product Adoption Rate | 78% |
| Revenue Growth | 45% |
| Operating Margin | 34% |
| Market Share | 23% |
| Customer Churn | 5% |


In [219]:
from IPython.display import display, Markdown
display(Markdown(out))

| Metric | Value |
|:--|--:|
| New User Acquisition Cost | 4300% |
| Customer Satisfaction | 92% |
| Employee Satisfaction | 87% |
| Product Adoption Rate | 78% |
| Revenue Growth | 45% |
| Operating Margin | 34% |
| Market Share | 23% |
| Customer Churn | 5% |

**Expected output**

| Metric | Value |
|:--|--:|
| Customer Satisfaction | 92% |
| Employee Satisfaction | 87% |
| Product Adoption Rate | 78% |
| Revenue Growth | 45% |
| User Acquisition Cost | 43.00 |
| Operating Margin | 34% |
| Market Share | 23% |
| Previous Customer Churn | 8% |
| Customer Churn | 5% |


In [220]:
print(out)

| Metric | Value |
|:--|--:|
| New User Acquisition Cost | 4300% |
| Customer Satisfaction | 92% |
| Employee Satisfaction | 87% |
| Product Adoption Rate | 78% |
| Revenue Growth | 45% |
| Operating Margin | 34% |
| Market Share | 23% |
| Customer Churn | 5% |


#### Example 2: Chain workflow to anonymise, translate and restyle text

In [179]:
anonymize_and_translate_steps = [
    """"Perform named entity recognition NER on the text, enclosing each entity in the appropriate tag.
    <example>
    Before: 'Mr. Smith went to London, where he met his wife Jane Smith and his partner Leonard Farley'
    After: 'Mr. <LASTNAME>Smith</LASTNAME> went to <CITY>London</CITY>, where he met his wife, <FIRSTNAME>Jane</FIRSTNAME><LASTNAME>Smith</LASTNAME>and his partner<FIRSTNAME>Leonard</FIRSTNAME><LASTNAME>Farley</LASTNAME>
    </example>
    """,
    
    """Anonymize the text, replacing each entity marked with a tag with a pseudonym.
    Make sure to preserve the meaning and content of sentence by replacing each entity with the same pseudonym.
    <example>
    Before: 'Mr. <LASTNAME>Smith</LASTNAME> went to <CITY>London</CITY>, where he met his wife, <FIRSTNAME>Jane</FIRSTNAME><LASTNAME>Smith</LASTNAME>and his partner<FIRSTNAME>Leonard</FIRSTNAME><LASTNAME>Farley</LASTNAME>,
    After: 'Mr. Frum went to Balberry,where he met with his wife, Ellen Frum, and his partner, Algernon Leiter.
    </example>
    """,
    """Translate the anonymized text in Italian.""",
    """Turn the Italian into verses""",
    """Turn the verse into romanesco, in the style of Belli""",
]

In [225]:
secretary_prompt = "You are a helpful secretary who values confidentiality and is adept at translations"
text = "It was a rainy day in Glasgow, when brothers Chad and John Smith embarked on their journey to Rome, where they hoped they would meet General Tso and his chicken"
chainer2 = Chainer(system_prompt=secretary_prompt, steps=anonymize_and_translate_steps, inp=text, model="gpt-4o", api_key=api_key)
tradit = chainer2.chain()

In [226]:
tradit

"Era 'na giornata de pioggia a Redford,  \nquanno li fratelli Percy e Rupert Frum,  \nse misero in viaggio, 'ndo và ssòrte er bordo,  \nverso Eldoria, 'n posto tutto fummo.  \n\nDove speraveno d'incontrà er comandante,  \ner Comandante Hawke, omo senza fallo,  \nco' er suo pollo, sempre accanto stante,  \n'n viaggio che sperano nun diventi callo."

#### Example 3: Chain workflow to create and perfect blog posts

In [238]:
class Blogger(Chainer):
    def __init__(self, topic, target_audience, word_count, model="gpt-4", api_key=None):
        # Define the system prompt for blog post generation
        system_prompt = """You are a professional blog content creator who specializes in creating
        high-quality, engaging blog posts on various topics. You follow SEO best practices
        and create content that is both informative and engaging."""
        
        # Define the steps for the blog post creation pipeline
        steps = [
            # Step 1: Generate an outline
            """Create a detailed outline for a blog post on the given topic.
            Include a title, introduction, 4-6 main sections with subpoints, and conclusion.
            Format the outline with clear headings and bullet points.""",
            
            # Step 2: Validate and refine the outline
            """Review the outline and ensure it meets these criteria:
            - Covers the topic comprehensively
            - Flows logically from point to point
            - Addresses the needs of the target audience
            - Includes specific, actionable information
            If any criteria are not met, refine the outline accordingly.""",
            
            # Step 3: Expand the outline into a complete post
            """Based on the validated outline, write the complete blog post.
            Include all of the following:
            1. An engaging title
            2. A hook-filled introduction
            3. Well-developed main sections with subheadings
            4. A strong conclusion with a call-to-action
            5. Write in a conversational but professional tone    
            The complete post should be approximately {word_count} words."""
        ]
        
        
        # Prepare input for the chaining process
        input_data = f"""
        Topic: {topic}
        Target Audience: {target_audience}
        Approximate Word Count: {word_count}
        """
        
        # Initialize the parent class
        super().__init__(system_prompt, steps, input_data, model, api_key)
    
    def generate_blog_post(self, verbose=True):
        """Generate a complete blog post using the chaining process"""
        print(f"🚀 Starting blog post generation process...")
        result = self.chain(verbose=verbose)
        print(f"✅ Blog post generation complete!")
        return result
    
    def save_to_file(self, filename):
        """Save the generated blog post to a markdown file"""
        blog_post = self.generate_blog_post(verbose=False)
        with open(filename, 'w') as f:
            f.write(blog_post)
        print(f"📝 Blog post saved to {filename}")

In [239]:
blogger = Blogger(
    topic="Philosophy of Logic",
    target_audience="advanced undergraduate and graduate students in Philosophy, Computer Science and Mathematics",
    word_count=1400,
    model="gpt-4o-mini",
    api_key=api_key
)
post = blogger.generate_blog_post()

🚀 Starting blog post generation process...
STEP: 0
# Outline for Blog Post: "The Philosophy of Logic: Foundations, Challenges, and Implications"

## Introduction
- Brief overview of logic’s integral role in philosophy, computer science, and mathematics.
- Importance of understanding the philosophical underpinnings of logic.
- Purpose of the blog post: To explore the philosophy of logic, its key components, challenges, and implications in various fields.

## I. Defining Logic and Its Philosophical Roots
### A. What is Logic?
   - Traditional definitions of logic.
   - Difference between formal and informal logic.
### B. Historical Context
   - Contributions of early philosophers: Aristotle’s syllogism, Stoic logic.
   - Development during the Middle Ages and the Renaissance.
   - Transition to modern logic: Frege, Russell, and Wittgenstein.

## II. Types of Logic
### A. Classical Logic
   - Overview of propositional and predicate logic.
   - Key principles: Law of non-contradiction, Law

In [235]:
# display(Markdown(post))

### Routing: Dynamically selects specialized LLM paths based on input characteristics

In [243]:
class LLMPrompt:
    def __init__(self, role, prompt_type, prompt_message):
        return {
            "role": role,
            "content": [{
                "type": prompt_type,
                prompt_type: prompt_message
            }]
        }
            
class LLMAgentSystemPrompt(LLMPrompt):
    def __init__(self, system_prompt):
        super().__init__(role="developer", prompt_type="text", prompt_message=system_prompt)


SyntaxError: incomplete input (1044036995.py, line 11)

In [ ]:
class LLMAgent:
    def __init__(self, system_prompt, model, api_key):
        self.client = OpenAI(api_key=api_key)
        self.model = model
        self.system_prompt = LLMAgentSystemPrompt(system_prompt)

In [ ]:
class Router(LLMAgent):
    gating_prompt: str
    choices: List[str]
    routes: List[str]

    def __init__(self, system_prompt, choices, routes, model, api_key):
        super().__init__(system_prompt, model, api_key)
        
    
    
    